# Progetto di Foundation of computer science

### Corso di laurea in Data Science A.A 2019/2020

### Simone D'Amico - 850369

In [ ]:
# Import delle libreire usate
import pandas as pd
import numpy as np

In [ ]:
# Caricamento del tempo di esecuzione di ogni cella
%load_ext autotime

# Definizione del formato per i numeri decimali, si mostra fino alla terza cifra decimale inclusa
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.options.mode.chained_assignment = None

### Caricamento dei dataset utilizzati

In [ ]:
# Per motivi di tempo e di memoria sono caricate dai dataset solo le colonne usate per i vari task

df_loans_lenders = pd.read_csv("loans_lenders.csv")

use_col_loans =["loan_id", "loan_name", "disburse_time", "planned_expiration_time", 
                "country_name", "loan_amount", "country_code",]
date_type = ["disburse_time", "planned_expiration_time"]
df_loans = pd.read_csv("loans.csv", usecols=use_col_loans, parse_dates=date_type)


use_col_lenders =["permanent_name", 'city', 'state', "country_code"]
df_lenders = pd.read_csv("lenders.csv", usecols=use_col_lenders)


used_col_country = ['country_name', 'country_code', 'population', 'population_below_poverty_line']
df_country_stats = pd.read_csv('country_stats.csv', usecols=used_col_country)

### 1. Normalize the loan_lenders table. In the normalized table, each row must have one `loan_id` and one lender.

In [ ]:
# Partendo dal dataset df_loans_lenders si usano come indici loan_id, così si conservano durante l'esecuzione. 
# Ogni valore di lenders viene trasformato in una lista e, grazie a explode, 
# si ottengono le coppie (loan, specifico lender)
df_loan_lender_norm = df_loans_lenders.set_index(['loan_id']).apply(
    lambda x: x.str.split(", ").explode()).reset_index()

In [ ]:
df_loan_lender_norm.head()

### 2. For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing.

In [ ]:
# Si calcola la differenza in giorni tra le colonne delle date e si considerano i giorni
df_loans["diff_days"] = (df_loans["planned_expiration_time"] - df_loans["disburse_time"]).dt.days

In [ ]:
df_loans.head()

In [ ]:
# Per verificare che le righe della colonna diff_days siano mancanti se almeno una delle colonne 
# planned_expiration_time o disburse_time è mancante si può usare la funzione info()

df_loans[df_loans['planned_expiration_time'].isnull()].info()

print("\n--------------------------------------------------------------\n")

df_loans[df_loans['disburse_time'].isnull()].info()

### 3. Find the lenders that have funded at least twice.

In [ ]:
# Si raggruppa per lender e si ottiene il numero di record di ogni gruppo
count = df_loan_lender_norm.groupby(['lenders']).size()

# Si selezionano solo i lenders che hanno finanziato due o più volte 
count = count[count >=2]

# Si definisce un dataset per mostrare il risultato
df_count = count.reset_index(name='count')

### 4. For each country, compute how many loans have involved that country as borrowers.

In [ ]:
# Si può usare value_count applicato alla colonna country_name per contare le occorrenze di ogni paese 
# ottienendo un dataframe 
df_count_loans = df_loans["country_name"].value_counts().reset_index()

df_count_loans = df_count_loans.rename(columns= {'index': 'country_name', 'country_name': 'count'})

In [ ]:
df_count_loans.head()

### 5. For each country, compute the overall amount of money borrowed.

In [ ]:
# Si raggruppano i loans per country_name e si calcola la somma di loan_amount per ogni gruppo
df_overall_borrowed = df_loans.groupby("country_name")["loan_amount"].sum().reset_index(name='tot_borrow')

In [ ]:
df_overall_borrowed.head()

### 6. Like the previous point, but expressed as a percentage of the overall amount lent.

In [ ]:
# Si calcola la somma totale dei prestiti
tot_borrow = df_overall_borrowed["tot_borrow"].sum()

# Si calcola la percentuale di prestiti di ogni stato rispetto alla somma totale
df_overall_borrowed["perc_borrow"] = (df_overall_borrowed["tot_borrow"]/tot_borrow)*100

In [ ]:
df_overall_borrowed.head()

### 7. Like the three previous points, but split for each year (with respect to disburse time).

In [ ]:
# Alcune osservazioni sono mancanti per la colonna disburse_time.
# Si vedono le proporzioni di quesi valori mancanti rispetto a tutto il dataset
# sia per il numero di righe che per l'ammontare del prestito a cui non è associato nessuna data.

df_missing_disburse_time = df_loans[df_loans['disburse_time'].isna()]

row_missing = df_missing_disburse_time.shape[0]
loan_amount_missing = df_missing_disburse_time['loan_amount'].sum()

# Osservazioni mancanti rispetto alle righe del dataset
print(f"Righe totali del dataset: {format(df_loans.shape[0], ',d')}")
print(f"Righe mancanti: {format(row_missing, ',d')}")
print(f"Percentuale di righe mancanti: {round(((row_missing/df_loans.shape[0])*100), 3)}%\n")

# Osservazioni mancanti rispetto ai prestiti
print(f"Totale prestiti: {format(df_loans['loan_amount'].sum(), ',.3f')}")
print(f"Prestiti senza anno: {format(loan_amount_missing, ',.3f')}")
print(f"Percentuale prestiti senza anno: {format(round(((loan_amount_missing/df_loans['loan_amount'].sum())*100), 3), '.3f')}%")

In [ ]:
# Dato l'esiguo numero di valori mancanti si è scelto di non operare nessuna sostituzione di tali valori
# ma di considerarli invece allo stesso livello dei gruppi degli anni.

# Si effettua il group by sulla data da cui è stato estratto l'anno. Si usa dt per gestire le date nelle 
# pandas Series. Per ogni gruppo si calcolano il numero di righe e la somma dei loan
df_group = df_loans.groupby(df_loans["disburse_time"].dt.year)["loan_amount"].agg([('count', 'count'), 
                                                                            ('loan_amount', 'sum')])

# Cast degli anni ad intero
df_group.index = df_group.index.astype(int)

# Al dataset finale si aggiunge una riga con le statistiche per le osservazioni mancanti.
df_group = df_group.append(pd.Series({'count':row_missing, 
                                      'loan_amount':loan_amount_missing}, name="missing_year"))

# Si calcola la percentuale di loan amount per ogni anno riferita alla somma totale dei loans
df_group["percentage_amount"] = (df_group["loan_amount"]/df_group["loan_amount"].sum())*100


df_group = df_group.reset_index()

In [ ]:
df_group.head()

### 8. For each lender, compute the overall amount of money lent. For each loan that has more than one lender, you must assume that all lenders contributed the same amount.

In [ ]:
# Calcolo, per ogni loan, del numero dei suoi partecipanti
df_n_lenders = df_loan_lender_norm.groupby('loan_id').count().reset_index()
df_n_lenders = df_n_lenders.rename(columns= {'lenders': 'n_lenders'})
df_n_lenders.head()

In [ ]:
# Calcolo, per ogni loan, del numero dei suoi partecipanti
df_n_lenders = df_loan_lender_norm.groupby('loan_id').count().reset_index()
df_n_lenders = df_n_lenders.rename(columns= {'lenders': 'n_lenders'})

# merge dei dataframe
df_merge = pd.merge(df_loans[['loan_id', 'loan_amount']], df_loan_lender_norm, on="loan_id")
df_merge = pd.merge(df_merge, df_n_lenders, on="loan_id")

# Calcolo dell'ammontare di ogni prestatore per ogni prestito in modo equo
df_merge['amount_per_lender'] = df_merge['loan_amount'] / df_merge["n_lenders"]

# Raggruppo per lender e calcolo la somma dei prestiti dati
df_groupby = df_merge.groupby("lenders")["amount_per_lender"].sum().reset_index()

In [ ]:
df_groupby.head()

### 9. For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.

In [ ]:
# Nel dataset c'è un numero elevato di dati mancanti per la colonna country_code
df_lenders.info(null_counts=True)

print("\n--------------------------------------------------------------\n")

# Inoltre si vede che nel dataset dei loans la colonna country_code ha 9 valori mancanti, 
# mentre country_name non ne ha 
df_loans.info()

In [ ]:
# I valori mancanti in df_loans sono associati tutti allo stato della Namibia
df_loans[df_loans['country_code'].isna()]

In [ ]:
# Analogo discorso per i record dei lenders, dove tutti i valori della Namibia sono mancanti
df_lenders[df_lenders['state']=='Namibia']

In [ ]:
# Per evitare di perdere tutte le informazioni legate ad un intero paese,
# solo in questo caso si procede con un rimpiazzo manuale dei missing value
df_lenders.loc[df_lenders['state']=='Namibia', 'country_code'] = 'Namibia'
df_loans.loc[df_loans['country_code'].isna(), 'country_code'] = 'Namibia'

In [ ]:
# Si separano i lenders con valore di country_code presente dai lenders con valore country_code mancante
df_lenders_no_na = df_lenders[df_lenders['country_code'].notna()]

# Si ottiene una serie di booleani per poter eseguire la sostistuzione dei valori mancanti successivamente
sr_lenders_na = df_lenders['country_code'].isna()

In [ ]:
# Si trova la distribuzione dei lender con valore di country_code presente 
# utilizzando l'opzione normalize di value_count che calcola la frequenza relativa dei valori
sr_state_distr = df_lenders_no_na['country_code'].value_counts(normalize=True)

# Si usa la distribuzione trovata per assegnare lo stato per i valori mancanti di country_code.
# Si usa la funzione choice utilizzando come valori i codici dei paesi, come dimensione del campione la somma dei
# booleani della serie che distingue tra codice na e non na e per probabilità la distribuzione trovata.
df_lenders.loc[sr_lenders_na, "country_code"] = np.random.choice(sr_state_distr.index, size=sum(sr_lenders_na), 
                                                                 p=sr_state_distr.values)

In [ ]:
# si verifico che non ci siano valori ancanti
sum(df_lenders.country_code.isna())

In [ ]:
# Si effettua il merge dei lenders con quanto hanno prestato in totale
df_lent = pd.merge(df_lenders, df_groupby, left_on='permanent_name', right_on='lenders')

In [ ]:
# Si sommano i prestiti raggruppati per paese
df_lent_country = df_lent.groupby('country_code')['amount_per_lender'].sum().reset_index(name='amount_lent')

In [ ]:
df_lent_country.head()

In [ ]:
# Si calcola il totale dei prestiti ricevuti per ogni paese
df_borrowed = df_loans.groupby('country_code')['loan_amount'].sum().reset_index(name='borrowed')

In [ ]:
df_borrowed.head()

In [ ]:
# Si effettua il merge dei due dataset
df_merge = pd.merge(df_lent_country, df_borrowed, on="country_code", how='outer')

# Si sostituiscono i valori mancanti con 0
df_merge.fillna(0, inplace=True)

# Si calcola la differenza tra il denaro prestato e quello preso in prestito
df_merge['diff_lent-borrowed'] = df_merge['amount_lent'] - df_merge['borrowed']

In [ ]:
df_merge.head()

### 10. Which country has the highest ratio between the difference computed at the previous point and the population?

In [ ]:
# Come nel punto precedente, il codice dello stato della Namibia è assente
df_country_stats[df_country_stats['country_name'] == 'Namibia']

In [ ]:
# Per non perdere le sue informazioni lo si inserisce manualmente
df_country_stats.loc[df_country_stats['country_name'] == 'Namibia', 'country_code'] = 'Namibia'

In [ ]:
# Si effettua il merge del dataset delle informazioni degli stati con quello della differenza di prestiti 
# calcolato al punto precedente
df_money_state = pd.merge(df_merge, df_country_stats, on="country_code")

# Calcolo del rapporto richiesto
df_money_state['ratio_diff_L-B_pop'] = df_money_state['diff_lent-borrowed']/df_money_state['population']

In [ ]:
df_money_state.head()

In [ ]:
# Si trova lo stato con il rapport più grande
df_money_state.iloc[[df_money_state['ratio_diff_L-B_pop'].idxmax()]]

### 11. Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?

In [ ]:
# La colonna population_below_poverty_line è espressa in percentuale,
# Si calcola la percentuale di persone sopra la soglia di poverà
df_money_state['population_above_poverty_line'] = 100 - df_money_state['population_below_poverty_line']

# Si calcola la quantità di popolazione al di sotto della fascia di povertà utilizzando la popolazione totale 
# e la percentuale calcolata sopra
pop_above_poverty_line = (df_money_state['population']*df_money_state['population_above_poverty_line'])/100

# Si calcola il rapport richiesto
df_money_state['ratio_diff_L-B_pop_above_poverty'] = df_money_state['diff_lent-borrowed']/pop_above_poverty_line

In [ ]:
df_money_state.head()

In [ ]:
# Si trova lo stato con il rapport più grande
df_money_state.iloc[[df_money_state['ratio_diff_L-B_pop_above_poverty'].idxmax()]]

### 12. For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year. For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD * 31 / (31+365+30) = 363.85 for 2016, 5000USD * 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.

Ci sono tre possibili casi da considerare:
1. i loans per cui le date di disburse_time e planned_expiration_time sono presenti e sono coerenti, 
   cioè quando planned_expiration_time > disburse_time
2. i loans in cui le due date sono presenti ma si ha planned_expiration_time > disburse_time
3. i loans per cui almeno una delle due date non è presente

Il primo passaggio consiste ne separare le righe del caso 3 dalle altre, per questi loans non è possibile 
calcolare i dati richiesti, si sostiuiranno i dati mancanti con 0.

Il passo successivo sarà distinguere i loans appartenenti al caso 1 da quelli appartenenti al caso 2,
nel primo caso si procederà per il calcolo dell'ammontere per anno.

Per i lenders che appartengono al caso 2, dato che non è possibile ricevere un prestito dopo la data della 
sua scadenza, si considerano i due valori come identici, in particolare uguali a disburse_time.

In [ ]:
# Si separano i lender con valori mancanti dagli altri
df_loans_na = df_loans[df_loans["disburse_time"].isna() | df_loans['planned_expiration_time'].isna()]
df_loans_no_na = df_loans.dropna(subset=["disburse_time", "planned_expiration_time"])

In [ ]:
# info dei due dataframe
df_loans_na.info()
print()
print("-------------------------------------------------------------")
print()
df_loans_no_na.info()

In [ ]:
# Si gestiscono i casi in cui le due date non sono corenti
loans_inc = df_loans_no_na['disburse_time'] > df_loans_no_na['planned_expiration_time']

df_loans_no_na.loc[loans_inc, 'planned_expiration_time'] = df_loans_no_na.loc[loans_inc, 'disburse_time']

In [ ]:
def get_amount_x_year(row):
    '''
    Riceve in input una riga del dataset e calcola, per ogni anno tra disburse_time e planned_expiration_time,
    la quantità di giorni di quell'anno. Nel caso di disburse_time considera i giorni dalla data fino alla fine 
    dell'anno, nel caso di planned_expiration_time considera i giorni passati dall'inizio dell'anno alla data in 
    questione. Ritorna un dizionario le cui chiavi sono gli anni all'interno del periodo considerato e i valori
    sono l'ammontare del loan per quell'anno dell'anno calcolati come descritto.
    
    ...

    Parameters
    ----------
    row : pandas.core.series.Series
        La riga del dataset
    
    
    Returns
    -------
    dict
        dizionario di con le informazioni del loan(id, name, amount), i giorni del prestito e per ogni anno
        l'importo in base ai giorni trascorsi
    '''
    
    # Si estraggono gli anni in mezzo
    planned = row["planned_expiration_time"]
    disburse = row["disburse_time"]
    
    # Calcolo degli anni tra disburse e planned esclusi loro stessi
    range_year = list(range(disburse.year + 1, planned.year))
    
    tot_amount = row['loan_amount']
    tot_days = (planned - disburse).days + 1
    
    amount_x_year = dict()
    
    # Si inseriscono i dati iniziali del prestito
    amount_x_year.update({
        'loan_id': row['loan_id'], 
        'loan_name': row['loan_name'], 
        'loan_amount': tot_amount,
        'days_loan': tot_days
    })
    
    # Se planned e disburse hanno lo stesso anno, considero solo la loro differenza in giorni
    if planned.year == disburse.year:
        amount_x_year[planned.year] = tot_amount
        
    # Altrimenti se gli anni sono differenti si aggiungono entrambi
    else:
        # Per disburse devo calcolare quelli che rimangono fino alla fine dell'anno,
        # Si calcolano i giorni totali dell'anno in questione che vengono tolti ai giorni dell'anno passati 
        # fino a disburse
        days_x_year = pd.Period("{}-12-31".format(disburse.year)).dayofyear - disburse.dayofyear
        
        # Calcolo dell'ammontare per l'anno disburse
        amount_x_year[disburse.year] = tot_amount*(days_x_year/tot_days)
        
        # Per planned si calcolano i giorni passati fino alla data in questione
        # e l'ammontare per l'anno planned
        amount_x_year[planned.year] = tot_amount*((planned.dayofyear)/tot_days)
        
        # Per gli anni all'interno del range aggiungo coppia (anno, ammontere per quell'anno) al dict
        if range_year:
            amount_x_year.update({
                k: tot_amount*(pd.Period("{}-12-31".format(k)).dayofyear/tot_days) for k in range_year
            })
            
    return amount_x_year


In [ ]:
# Calcolo del prestito per ogni anno
res = df_loans_no_na.apply(lambda x: get_amount_x_year(x), axis=1)

In [ ]:
# Creazione del dataframe e riordino delle colonne
df_res = pd.DataFrame(list(res))
df_res.columns = list(map(str, df_res.columns))
cols_reorder = ['loan_id', 'loan_name', 'loan_amount', 'days_loan', '2011', 
                '2012', '2013', '2014', '2015', '2016', '2017', '2018']
df_res = df_res.reindex(cols_reorder, axis=1)


In [ ]:
df_res.head()

In [ ]:
# Gestione dei record con almeno una delle due date mancanti, si creano le colonne degli anni e dei giorni passati
# che contengono solo 0
df_na_sel = df_loans_na[['loan_id', 'loan_name','loan_amount']]
df_na_sel[['days_loan', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018']] = pd.DataFrame(
    [np.full([9], np.nan)], index=df_na_sel.index)

In [ ]:
# Concatenazione dei due dataset
df_res = pd.concat([df_res, df_na_sel])

In [ ]:
df_res.head()

In [ ]:
df_res[['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018']].sum().reset_index(name ='Total Amount')